In [15]:
#je vais me connecter à la base de données annonces.db faire des prédictions des prix grace à un modèle de machine learning
#je vais ensuite enregistrer le modèle dans un fichier pour pouvoir l'utiliser plus tard
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np



In [16]:
#je me connecte à la base de données
conn = sqlite3.connect("annonces.db")
#je récupère les données
data = pd.read_sql_query("SELECT * FROM annonces", conn)
#je ferme la connexion
conn.close()

In [17]:
# je vais faire un boxplot pour voir si il y a des valeurs aberrantes
print(data.head())

# je vais supprimer les valeurs aberrantes
# data = data[data["price"] < 1000000]


   id  location  pieces  surface   price  \
0   1        10       1     15.4   740.0   
1   2         8       1     40.0  2000.0   
2   3        16       1     12.0   585.0   
3   4         1       3     73.0  3499.0   
4   5         5       2     38.0  1610.0   

                                         description  
0  Studio refait à neuf très belle prestation rez...  
1  **Idéal étudiant(e)** À 100 m du Parc Monceau,...  
2  **Disponible fin octobre 2024** Studette meubl...  
3  Très bel appartement parisien de 73m2, entière...  
4  **C'est ma résidence principale que je loue pe...  


In [18]:
import matplotlib.pyplot as plt
data.columns = data.columns.str.strip()
print(data.columns.tolist())

data = data[data["price"] < 3500]

#je crée un boxplot pour voir si il y a des valeurs aberrantes
data.boxplot(column="price")

['id', 'location', 'pieces', 'surface', 'price', 'description']


<Axes: >

In [19]:
from sklearn.ensemble import RandomForestRegressor


X = data[['surface','location','pieces']]
y = data['price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création du modèle de régression des forêts aléatoires
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Affichage des résultats
results = pd.DataFrame({'Location': X_test['location'],'Surface': X_test['surface'],  
                        'Prix Réel': y_test, 'Prix Prédit': y_pred})
print(results)

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Calcul des erreurs
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# MAPE - Erreur moyenne absolue en pourcentage
mape = (abs((y_test - y_pred) / y_test).mean()) * 100

# Affichage des résultats
print(f'Coefficient de détermination (R²): {r2}')
print(f'Erreur quadratique moyenne (MSE): {mse}')
print(f'Racine carrée de l\'erreur quadratique moyenne (RMSE): {rmse}')
print(f'Erreur absolue moyenne (MAE): {mae}')
print(f'Erreur moyenne absolue en pourcentage (MAPE): {mape:.2f}%')


     Location  Surface  Prix Réel  Prix Prédit
26         11     37.0     1600.0   769.370000
117        15     30.0     1350.0  1408.208095
81         15     80.0     2750.0  2746.840000
209        17     74.0     2500.0  2751.640000
60         13     40.0     1360.0  1567.790000
..        ...      ...        ...          ...
316        18     60.0     2100.0  2032.670000
312        15     34.0     1650.0  1489.451429
124        15     48.0     1860.0  1891.140000
83         15     69.0     2490.0  2440.960000
259        18    105.0     2995.0  2213.380000

[67 rows x 4 columns]
Coefficient de détermination (R²): 0.7729962132159078
Erreur quadratique moyenne (MSE): 114341.94940061509
Racine carrée de l'erreur quadratique moyenne (RMSE): 338.1448645190624
Erreur absolue moyenne (MAE): 226.3963361762615
Erreur moyenne absolue en pourcentage (MAPE): 15.79%


In [20]:
#je vais le faire avec un modèle de régression linéaire
#je vais créer un modèle de régression linéaire
model = LinearRegression()
#je vais entrainer mon modèle
model.fit(X_train, y_train)
#je vais faire des prédictions
y_pred = model.predict(X_test)
#je vais afficher les résultats
results = pd.DataFrame({'Location': X_test['location'],'Surface': X_test['surface'], 
                        'Prix Réel': y_test, 'Prix Prédit': y_pred})
print(results)
#je vais calculer les erreurs
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
#je vais afficher les résultats
print(f'Coefficient de détermination (R²): {r2}')
print(f'Erreur quadratique moyenne (MSE): {mse}')
print(f'Racine carrée de l\'erreur quadratique moyenne (RMSE): {rmse}')
print(f'Erreur absolue moyenne (MAE): {mae}')


     Location  Surface  Prix Réel  Prix Prédit
26         11     37.0     1600.0  1535.866394
117        15     30.0     1350.0  1259.775951
81         15     80.0     2750.0  2669.954408
209        17     74.0     2500.0  2407.225375
60         13     40.0     1360.0  1565.493849
..        ...      ...        ...          ...
316        18     60.0     2100.0  2022.204259
312        15     34.0     1650.0  1362.959137
124        15     48.0     1860.0  1724.100284
83         15     69.0     2490.0  2326.006328
259        18    105.0     2995.0  3243.209413

[67 rows x 4 columns]
Coefficient de détermination (R²): 0.8625140617385805
Erreur quadratique moyenne (MSE): 69251.75310372825
Racine carrée de l'erreur quadratique moyenne (RMSE): 263.15727826478263
Erreur absolue moyenne (MAE): 188.34968023499871


In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Création du modèle avec des hyperparamètres ajustés
model = MLPRegressor(hidden_layer_sizes=(96, 64, 32), max_iter=2000, random_state=42)

# Entraînement du modèle
model.fit(X_train_scaled, y_train)

# Prédictions
y_pred = model.predict(X_test_scaled)

# Affichage des résultats
results = pd.DataFrame({'Location': X_test['location'], 'Surface': X_test['surface'], 
                        'Prix Réel': y_test, 'Prix Prédit': y_pred})
print(results)

# Calcul des erreurs
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'Coefficient de détermination (R²): {r2}')
print(f'Erreur quadratique moyenne (MSE): {mse}')
print(f'Racine carrée de l\'erreur quadratique moyenne (RMSE): {rmse}')
print(f'Erreur absolue moyenne (MAE): {mae}')

# Enregistrement du modèle
joblib.dump(model, 'mlp_model.pkl')


     Location  Surface  Prix Réel  Prix Prédit
26         11     37.0     1600.0  1493.873894
117        15     30.0     1350.0  1257.029620
81         15     80.0     2750.0  2556.567343
209        17     74.0     2500.0  2378.381393
60         13     40.0     1360.0  1560.019949
..        ...      ...        ...          ...
316        18     60.0     2100.0  2033.994584
312        15     34.0     1650.0  1372.575255
124        15     48.0     1860.0  1750.281571
83         15     69.0     2490.0  2263.402655
259        18    105.0     2995.0  3142.320507

[67 rows x 4 columns]
Coefficient de détermination (R²): 0.8652427527531671
Erreur quadratique moyenne (MSE): 67877.30973280544
Racine carrée de l'erreur quadratique moyenne (RMSE): 260.53274215116505
Erreur absolue moyenne (MAE): 185.31771322045034


['mlp_model.pkl']